In [2]:
!pip install transformers
!pip install sentencepiece
from transformers import MarianMTModel, MarianTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.8 MB/s eta 0:00:00


In [3]:
#EN to ES
first_model_name = 'Helsinki-NLP/opus-mt-en-es'
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)
first_model = MarianMTModel.from_pretrained(first_model_name)

# ES to EN
second_model_name = 'Helsinki-NLP/opus-mt-es-en'
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)
second_model = MarianMTModel.from_pretrained(second_model_name)

#EN to ZH
third_model_name = 'Helsinki-NLP/opus-mt-en-zh'
third_model_tkn = MarianTokenizer.from_pretrained(third_model_name)
third_model = MarianMTModel.from_pretrained(third_model_name)

# ZH to EN
fourth_model_name = 'Helsinki-NLP/opus-mt-zh-en'
fourth_model_tkn = MarianTokenizer.from_pretrained(fourth_model_name)
fourth_model = MarianMTModel.from_pretrained(fourth_model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
# add tokens before original texts
def format_batch_texts(language_code, batch_texts):

  formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]

  return formated_bach

In [5]:
def translate(batch_texts, model, tokenizer, language="es"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)

    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    return translated_texts

In [6]:
#cleaning up text
import re
def clean_sentence(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    #line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm1234567890 ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    #print(clean_line)
    if clean_line:
      if clean_line[0] == ' ':
          clean_line = clean_line[1:]
    return clean_line

In [7]:
import re
# split long context into chunks
def split_text(text, limit):
    textArr = []
    textLen = len(text)
    if textLen > limit:
        for i in range(0, textLen, limit):
            textArr.append(text[i:i + limit])
    else:
        textArr.append(text)
    return textArr

# back translate, taking into consideration really long texts
def back_translate(batch_texts, src="en", tmp="es"):
    final_translated_batch = []

    for text in batch_texts:
        # Split the text into 100-word chunks
        chunks = split_text(text,100)

        # Translate each chunk individually
        tmp_translated_chunks = [translate([chunk], first_model, first_model_tkn, tmp)[0] for chunk in chunks]
        back_translated_chunks = [translate([chunk], second_model, second_model_tkn, src)[0] for chunk in tmp_translated_chunks]

        # Merge the translated chunks back together
        final_translated_text = ' '.join(back_translated_chunks)

        # Some short translated sentence starts with "- " so need to check and remove
        if final_translated_text.startswith("- "):
            final_translated_text = final_translated_text[2:]
        elif final_translated_text.startswith("— "):
            final_translated_text = final_translated_text[2:]

        final_translated_batch.append(final_translated_text)

    return final_translated_batch

In [8]:
# write translated data to context, question and answer cell
def write_trans_data(dataset, src, tmp):
    trans_format = []
    n_translated = 0
    out_of_context = 0
    in_context = 0
    for instance in dataset["data"]:
        n_translated += 1
        print("Translated:",n_translated)
        # translate no more than 250 instances for computational efficiency
        if n_translated > 250:
            break
        # translate context
        context = [instance['paragraphs'][0]['context']]
        #print("context: ",context)
        trans_context = back_translate(context, src, tmp)
        trans_context = clean_sentence(" ".join(trans_context))

        trans_instance = {
            "title": instance['title'],
            "paragraphs": [{
                'context': trans_context,
                'qas': []
            }]
        }

        for qa in instance['paragraphs'][0]['qas']:
            # capture multiple answers in the same question
            question = [qa['question']]
            #print("question:",question)
            trans_question = back_translate(question, src, tmp)
            trans_question = clean_sentence(" ".join(trans_question))
            # update to aug_instance
            new_qa = {
                "question": trans_question,
                "id": qa['id'],
                "answers": []
            }
            for ans in qa['answers']:
                #print("type of ans['text'] is: ", type(ans['text']))
                #print("ans text input",ans['text'])
                trans_ans = back_translate([ans['text']], src, tmp)
                trans_ans = clean_sentence(" ".join(trans_ans))
                #print(trans_ans)
                start_logit = trans_context.find(trans_ans)

                # Drop example if out of context
                # check if context/question/ans empty
                if start_logit == -1 or trans_ans=='' or trans_question =='' or trans_context=='':
                  out_of_context += 1
                else:
                  in_context += 1
                  new_qa["answers"].append({
                      'answer_start': start_logit,
                      'text': trans_ans
                  })
            if new_qa["answers"]:
              trans_instance["paragraphs"][0]['qas'].append(new_qa)
        trans_format.append(trans_instance)
    print("In context qa",in_context,"Out of context qa",out_of_context)
    return {"data":trans_format}

In [9]:
def export_to_json(dataset, data_dir, file_name):
    with open(f'{data_dir}/{file_name}.json', 'w') as outfile:
        json.dump(dataset, outfile)

In [10]:
# Experiment 1: back translate oodomain training data to es
import json
duorc = json.load(open(f'/content/drive/MyDrive/robustqa/datasets/oodomain_train/duorc'))
duorc_es = write_trans_data(duorc,'en','es')
export_to_json(duorc_es, "/content/drive/MyDrive/robustqa/datasets/oodomain_train", "duorc_es")

race = json.load(open(f'/content/drive/MyDrive/robustqa/datasets/oodomain_train/race'))
race_es = write_trans_data(race,'en','es')
export_to_json(race_es, "/content/drive/MyDrive/robustqa/datasets/oodomain_train", "race_es")

rex = json.load(open(f'/content/drive/MyDrive/robustqa/datasets/oodomain_train/relation_extraction'))
rex_es = write_trans_data(rex,'en','es')
export_to_json(rex_es, "/content/drive/MyDrive/robustqa/datasets/oodomain_train", "rex_es")

Translated: 1


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translated: 2
Translated: 3
Translated: 4
Translated: 5
Translated: 6
Translated: 7
Translated: 8
Translated: 9
Translated: 10
Translated: 11
Translated: 12
In context qa 102 Out of context qa 63
Translated: 1
Translated: 2
Translated: 3
Translated: 4
Translated: 5
Translated: 6
Translated: 7
Translated: 8
Translated: 9
Translated: 10
Translated: 11
Translated: 12
Translated: 13
Translated: 14
Translated: 15
Translated: 16
Translated: 17
Translated: 18
Translated: 19
Translated: 20
Translated: 21
Translated: 22
Translated: 23
Translated: 24
Translated: 25
Translated: 26
Translated: 27
Translated: 28
Translated: 29
Translated: 30
Translated: 31
Translated: 32
Translated: 33
Translated: 34
Translated: 35
Translated: 36
Translated: 37
Translated: 38
Translated: 39
Translated: 40
Translated: 41
Translated: 42
Translated: 43
Translated: 44
Translated: 45
Translated: 46
Translated: 47
Translated: 48
Translated: 49
Translated: 50
Translated: 51
Translated: 52
Translated: 53
Translated: 54
Tra

In [10]:
# Experiment 2: back translate to Chinese
import json
duorc = json.load(open(f'/content/drive/MyDrive/robustqa/datasets/oodomain_train/duorc'))
duorc_es = write_trans_data(duorc,'en','zh')
export_to_json(duorc_es, "/content/drive/MyDrive/robustqa/datasets/oodomain_train", "duorc_zh")

race = json.load(open(f'/content/drive/MyDrive/robustqa/datasets/oodomain_train/race'))
race_es = write_trans_data(race,'en','zh')
export_to_json(race_es, "/content/drive/MyDrive/robustqa/datasets/oodomain_train", "race_zh")

rex = json.load(open(f'/content/drive/MyDrive/robustqa/datasets/oodomain_train/relation_extraction'))
rex_es = write_trans_data(rex,'en','zh')
export_to_json(rex_es, "/content/drive/MyDrive/robustqa/datasets/oodomain_train", "rex_zh")

Translated: 1


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translated: 2
Translated: 3
Translated: 4
Translated: 5
Translated: 6
Translated: 7
Translated: 8
Translated: 9
Translated: 10
Translated: 11
Translated: 12
In context qa 102 Out of context qa 63
Translated: 1
Translated: 2
Translated: 3
Translated: 4
Translated: 5
Translated: 6
Translated: 7
Translated: 8
Translated: 9
Translated: 10
Translated: 11
Translated: 12
Translated: 13
Translated: 14
Translated: 15
Translated: 16
Translated: 17
Translated: 18
Translated: 19
Translated: 20
Translated: 21
Translated: 22
Translated: 23
Translated: 24
Translated: 25
Translated: 26
Translated: 27
Translated: 28
Translated: 29
Translated: 30
Translated: 31
Translated: 32
Translated: 33
Translated: 34
Translated: 35
Translated: 36
Translated: 37
Translated: 38
Translated: 39
Translated: 40
Translated: 41
Translated: 42
Translated: 43
Translated: 44
Translated: 45
Translated: 46
Translated: 47
Translated: 48
Translated: 49
Translated: 50
Translated: 51
Translated: 52
Translated: 53
Translated: 54
Tra

In [11]:
# Experiment 3: back translation + synonym replacement
duorc_sr_bt = json.load(open(f'/content/drive/MyDrive/robustqa/datasets/oodomain_train/duorc_SR_0.3&9.json'))
duorc_sr_bt = write_trans_data(duorc_sr_bt,'en','es')
export_to_json(duorc_sr_bt, "/content/drive/MyDrive/robustqa/datasets/oodomain_train", "duorc_sr_bt")

race_sr_bt = json.load(open(f'/content/drive/MyDrive/robustqa/datasets/oodomain_train/race_SR_0.3&9.json'))
race_sr_bt = write_trans_data(race_sr_bt,'en','es')
export_to_json(race_sr_bt, "/content/drive/MyDrive/robustqa/datasets/oodomain_train", "race_sr_bt")

rex_sr_bt = json.load(open(f'/content/drive/MyDrive/robustqa/datasets/oodomain_train/re_SR_0.3&9.json'))
rex_sr_bt = write_trans_data(rex_sr_bt,'en','es')
export_to_json(rex_sr_bt, "/content/drive/MyDrive/robustqa/datasets/oodomain_train", "rex_sr_bt")

Translated: 1
Translated: 2
Translated: 3
Translated: 4
Translated: 5
Translated: 6
Translated: 7
Translated: 8
Translated: 9
Translated: 10
Translated: 11
Translated: 12
Translated: 13
Translated: 14
Translated: 15
Translated: 16
Translated: 17
Translated: 18
Translated: 19
Translated: 20
Translated: 21
Translated: 22
Translated: 23
Translated: 24
Translated: 25
Translated: 26
Translated: 27
Translated: 28
Translated: 29
Translated: 30
Translated: 31
Translated: 32
Translated: 33
Translated: 34
Translated: 35
Translated: 36
Translated: 37
Translated: 38
Translated: 39
Translated: 40
Translated: 41
Translated: 42
Translated: 43
Translated: 44
Translated: 45
Translated: 46
Translated: 47
Translated: 48
Translated: 49
Translated: 50
Translated: 51
Translated: 52
Translated: 53
Translated: 54
Translated: 55
Translated: 56
Translated: 57
Translated: 58
Translated: 59
Translated: 60
Translated: 61
Translated: 62
Translated: 63
Translated: 64
Translated: 65
Translated: 66
Translated: 67
Tran